<a href="https://colab.research.google.com/github/hucarlos08/GEE-CIMAT/blob/main/LSU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Linear Spectral Unmixing (LSU)

Linear Spectral Unmixing is a technique used to address the common challenge in remote sensing known as the "mixed pixel" problem. Pixels in satellite images often cover areas on the ground containing multiple distinct surface materials (e.g., a mix of grass, soil, and pavement). LSU aims to **estimate the fractional abundance** (or proportion) of predefined fundamental surface materials, called **endmembers**, within each pixel.

## The Linear Mixing Model

The core assumption behind *linear* unmixing is that the spectrum measured by the sensor for a mixed pixel is a **linear combination** of the "pure" spectra of the endmembers present within that pixel, weighted by their respective fractional abundances.

Mathematically, for a given pixel, its reflectance $p_{\lambda}$ in a specific spectral band $\lambda$ is modeled as the sum of the endmember reflectances $m_{i, \lambda}$ weighted by their fractions $f_i$, plus some error $\epsilon_{\lambda}$:

$$ p_{\lambda} = \sum_{i=1}^{N} (f_i \cdot m_{i, \lambda}) + \epsilon_{\lambda} $$

Where:
*   $p_{\lambda}$: The reflectance measured for the pixel in band $\lambda$.
*   $N$: The number of endmembers.
*   $f_i$: The fractional abundance (proportion) of the *i*-th endmember within the pixel (the value we want to estimate).
*   $m_{i, \lambda}$: The reflectance of the *i*-th "pure" endmember material in band $\lambda$.
*   $\epsilon_{\lambda}$: The residual error in band $\lambda$, accounting for model inaccuracies and noise.

Considering all spectral bands simultaneously, this can be expressed more compactly using vector/matrix notation:

$$ \mathbf{p} = \mathbf{M} \mathbf{f} + \mathbf{e} $$

Where:
*   $\mathbf{p}$: The pixel spectrum vector (containing $p_{\lambda}$ for all bands).
*   $\mathbf{M}$: The matrix whose columns are the endmember spectra vectors ($\mathbf{m}_i$).
*   $\mathbf{f}$: The vector of fractional abundances ($f_i$) for all endmembers.
*   $\mathbf{e}$: The error vector (containing $\epsilon_{\lambda}$ for all bands).

## Endmembers: The Key Ingredient

The success of LSU heavily depends on identifying appropriate **endmembers**. These represent the pure, fundamental materials assumed to mix within the scene. Common endmembers might include:

*   Green Vegetation
*   Non-Photosynthetic Vegetation (Dry Grass, Wood)
*   Soil (various types)
*   Water (clear, turbid)
*   Impervious Surfaces (concrete, asphalt, roofing)
*   Shade/Shadow (often included as a "photometric" endmember)

**Defining Endmember Spectra:** This is often the most challenging step. Endmember spectra can be obtained from:
1.  **Spectral Libraries:** Standardized lab or field measurements (e.g., USGS Spectral Library, ECOSTRESS). Requires careful matching to image conditions and sensor bands.
2.  **Image-Based Methods:** Identifying the "purest" pixels within the image itself. This can be done manually by selecting pixels visually representing pure materials, or using automated techniques like the Pixel Purity Index (PPI) often combined with N-FINDR algorithm (more advanced).

## The `unmix()` Function in GEE

Google Earth Engine provides a convenient function, `image.unmix()`, that performs constrained linear spectral unmixing. You provide:
1.  The input image (typically Surface Reflectance).
2.  A list containing the endmember spectra (where each endmember's spectrum is itself a list of reflectance values corresponding to the input image bands).

The `unmix()` function solves the linear mixing equation for the fractional abundances ($\mathbf{f}$) for each pixel. Importantly, it applies constraints:
*   **Sum-to-One Constraint:** The sum of the estimated fractions for a pixel should ideally equal 1 (i.e., $\sum_{i=1}^{N} f_i = 1$).
*   **Non-Negativity Constraint:** Fractional abundances cannot be negative (i.e., $f_i \ge 0$ for all $i$).

The output is a multi-band image where each band represents the estimated fractional abundance ($f_i$) of the corresponding endmember provided in the input list.

## Connection to Matrix Algebra

Although the `unmix()` function abstracts the details, solving the equation $\mathbf{p} = \mathbf{M} \mathbf{f}$ for $\mathbf{f}$ fundamentally involves matrix algebra. In an unconstrained least-squares sense, it requires calculating the **pseudo-inverse** ($\mathbf{M}^{+}$) of the endmember matrix $\mathbf{M}$ and multiplying it by the pixel spectrum $\mathbf{p}$:

$$ \mathbf{f} \approx \mathbf{M}^{+} \mathbf{p} $$

The constraints (sum-to-one, non-negative) add complexity, often requiring iterative optimization or specific algorithms like Fully Constrained Least Squares (FCLS), which are handled internally by `image.unmix()`.

## Use Cases

*   **Urban Mapping:** Estimating the fraction of impervious surfaces vs. vegetation cover.
*   **Vegetation Monitoring:** Quantifying vegetation cover changes, separating green from dry vegetation.
*   **Water Body Analysis:** Estimating water fractions, potentially sediment load if suitable endmembers are used.
*   **Burn Severity Mapping:** Unmixing char, green vegetation, and soil components.
*   **Agriculture:** Assessing crop residue cover.

In [6]:
# -*- coding: utf-8 -*-
"""
This script demonstrates Linear Spectral Unmixing (LSU) using Google Earth Engine (GEE).
The goal is to estimate the fractional cover of basic land cover types (endmembers)
within each pixel of a Landsat 8 image.

We will:
1. Load and prepare a Landsat 8 Surface Reflectance image.
2. Define endmembers (Water, Vegetation, Soil/Urban) by selecting 'pure' pixels directly from the image.
3. Extract the spectral signatures (reflectance values across bands) for these endmembers.
4. Use the ee.Image.unmix() function to calculate the fraction of each endmember in every pixel.
5. Visualize the resulting fractional cover maps.
"""

# Import necessary libraries
import ee
import folium

# Helper function to add GEE tile layers to a Folium map
def add_ee_layer(self, ee_image_object, vis_params, name):
  """Adds a Google Earth Engine tile layer to a Folium map."""
  if ee_image_object is None:
      print(f"Warning: Skipping layer '{name}' because the Earth Engine Image object is null.")
      return self
  try:
      # --- Use the standard getMapId method ---
      map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
      # -----------------------------------------

      folium.raster_layers.TileLayer(
          tiles=map_id_dict['tile_fetcher'].url_format,
          attr='Map Data © <a href="https://earthengine.google.com/">Google Earth Engine</a>',
          name=name,
          overlay=True,
          control=True
      ).add_to(self)
      print(f"Layer '{name}' added successfully.")
  except Exception as e:
      print(f"ERROR adding layer '{name}': {e}")
      # Consider adding more detailed error printing if problems persist
      # import traceback
      # print(traceback.format_exc())
  return self

# Re-apply the corrected function to the folium.Map class
folium.Map.add_ee_layer = add_ee_layer

print("Libraries imported and CORRECTED helper function defined.") # Add note

print("Libraries imported and helper function defined.")

# -----------------------------------------------------------------------------
# Authenticate and Initialize Google Earth Engine
# -----------------------------------------------------------------------------
try:
    ee.Initialize()
    print("Google Earth Engine initialized successfully.")
except Exception as e:
    print(f"Earth Engine initialization failed: {e}")
    print("Attempting authentication...")
    ee.Authenticate()
    ee.Initialize(project='ee-cimat')
    print("Google Earth Engine authenticated and initialized successfully.")

# -----------------------------------------------------------------------------
# Configuration
# -----------------------------------------------------------------------------

# 1. Define Area of Interest (AOI)
# Let's use the San Francisco Bay Area again - good mix of water, vegetation, urban.
aoi = ee.Geometry.Rectangle([-123.0, 37.3, -121.8, 38.0])
# Center point for map display
map_center_coords = aoi.centroid().coordinates().getInfo()[::-1]
zoom_level = 9
print(f"AOI defined for San Francisco Bay area.")

# 2. Define Time Period
# Choose a period with likely clear imagery (e.g., late summer).
start_date = '2022-08-01'
end_date = '2022-09-30'
print(f"Time period set from {start_date} to {end_date}.")

# 3. Select Landsat Collection and Bands
landsat_collection_id = 'LANDSAT/LC08/C02/T1_L2' # Landsat 8 Collection 2 Surface Reflectance
# Bands for Unmixing: We'll use the 6 optical bands
# Blue, Green, Red, NIR, SWIR1, SWIR2
lsu_bands = ['SR_B2', 'SR_B3', 'SR_B4', 'SR_B5', 'SR_B6', 'SR_B7']
# Bands for True Color visualization
rgb_bands = ['SR_B4', 'SR_B3', 'SR_B2'] # Standard Red, Green, Blue
print(f"Using Landsat Collection: {landsat_collection_id}")
print(f"Bands for LSU: {lsu_bands}")
print(f"Bands for RGB: {rgb_bands}")

# -----------------------------------------------------------------------------
# Data Loading and Preprocessing
# -----------------------------------------------------------------------------

# Function to apply scaling factors for Landsat Collection 2 SR data
def apply_scale_factors(image):
  """Applies scaling factors to Landsat Collection 2 SR image."""
  # Apply scaling factors to optical bands (0.0000275 * DN - 0.2)
  optical_bands = image.select('SR_B.').multiply(0.0000275).add(-0.2)
  # Apply scaling factor to thermal bands (if needed, though not for LSU)
  # thermal_bands = image.select('ST_B.*').multiply(0.00341802).add(149.0)
  # Add scaled bands back, overwriting the originals
  # Also copy original properties (like system:time_start, CLOUD_COVER)
  return image.addBands(optical_bands, None, True)\
              .copyProperties(image, image.propertyNames()) # Ensure properties are copied

# Load the collection, filter, scale, and create a median composite
print("Loading and preparing image...")
landsat_composite = (ee.ImageCollection(landsat_collection_id)
                     .filterBounds(aoi)
                     .filterDate(start_date, end_date)
                     .filter(ee.Filter.lt('CLOUD_COVER', 10)) # Filter for low cloud cover
                     .map(apply_scale_factors) # Apply scaling factors to each image
                     .median() # Create a single composite image
                    )

# Select only the bands needed for LSU from the composite
image_lsu_input = landsat_composite.select(lsu_bands)

# --- Basic check if image is valid ---
try:
    input_band_names = image_lsu_input.bandNames().getInfo()
    if not all(b in input_band_names for b in lsu_bands) or len(input_band_names) == 0:
        print("ERROR: Input image is missing required LSU bands or is empty.")
        exit()
    print("Input image prepared with bands:", input_band_names)
except Exception as e:
    print(f"ERROR: Could not create or verify input image: {e}")
    exit()

# -----------------------------------------------------------------------------
# Define Endmembers (Image-Based Method)
# -----------------------------------------------------------------------------
# We will manually select points representing relatively "pure" examples of
# our desired endmembers directly from the image.

print("Defining endmember locations...")

# 1. Water Endmember: Choose a point in a deep, clear water body
water_point = ee.Geometry.Point([-122.3, 37.6]) # Example: SF Bay water
print("Water endmember point selected.")

# 2. Vegetation Endmember: Choose a point in a densely vegetated area (e.g., forest)
veg_point = ee.Geometry.Point([-122.5, 37.8]) # Example: Marin Headlands park area
print("Vegetation endmember point selected.")

# 3. Soil/Urban Endmember: Choose a point representing bare soil or dense urban/impervious area
# This can be tricky; aim for something spectrally distinct from vegetation and water.
# Sometimes requires averaging a few points or using a polygon. Let's try an urban area.
urban_soil_point = ee.Geometry.Point([-122.4, 37.75]) # Example: Dense urban area in SF

# Alternatively, find a bare soil field if visible:
# urban_soil_point = ee.Geometry.Point([-121.9, 37.4]) # Example: Area that might be fallow fields
print("Soil/Urban endmember point selected.")

# --- Extract Endmember Spectra ---
# We need to get the average reflectance values for the lsu_bands at these points.
scale = 30 # Native resolution for Landsat optical bands

print("Extracting endmember spectra...")

# Function to extract spectrum (mean reflectance) for a given point
def get_spectrum(point, image, bands):
    """Extracts mean reflectance values for specified bands at a point."""
    # Use reduceRegion to get the mean value of the pixel(s) at the point
    spectrum_dict = image.reduceRegion(
        reducer=ee.Reducer.mean(),
        geometry=point,
        scale=scale,
        # crs='EPSG:4326' # Optional: Specify CRS if needed, usually handles automatically
        # maxPixels=10 # Only need one pixel or a few if using a buffer
    ).getInfo() # Use getInfo() to bring the results client-side

    # Check if the dictionary contains the expected bands
    if not spectrum_dict or not all(b in spectrum_dict for b in bands):
        print(f"ERROR: Failed to extract valid spectrum at point {point.getInfo()['coordinates']}. Check point location and image data.")
        return None # Return None if extraction failed

    # Extract values in the *exact order* of the lsu_bands list
    spectrum_list = [spectrum_dict.get(band) for band in bands]
    return spectrum_list

# Extract spectra for each endmember
water_spectrum = get_spectrum(water_point, image_lsu_input, lsu_bands)
veg_spectrum = get_spectrum(veg_point, image_lsu_input, lsu_bands)
urban_soil_spectrum = get_spectrum(urban_soil_point, image_lsu_input, lsu_bands)

# Check if all spectra were extracted successfully
if None in [water_spectrum, veg_spectrum, urban_soil_spectrum]:
    print("ERROR: Failed to extract one or more endmember spectra. Halting execution.")
    exit()

# --- Assemble the Endmember List ---
# The unmix function needs a list of lists, where each inner list is an endmember spectrum.
# The order here defines the order of the output bands ('band_0', 'band_1', 'band_2').
# Let's use the order: [Water, Vegetation, Soil/Urban]
endmembers_list = [water_spectrum, veg_spectrum, urban_soil_spectrum]
endmember_names = ['Water', 'Vegetation', 'Soil_Urban'] # For reference

print("\nEndmember Spectra Extracted (Order: Water, Vegetation, Soil/Urban):")
for i, name in enumerate(endmember_names):
    # Format numbers for slightly better readability
    formatted_spectrum = [f"{val:.4f}" if val is not None else 'N/A' for val in endmembers_list[i]]
    print(f"- {name}: {formatted_spectrum}")

# IMPORTANT: Double-check that the spectra look reasonable!
# - Water: Low reflectance, maybe slightly higher in Blue/Green.
# - Vegetation: Low in Blue/Red, high peak in NIR, lower in SWIR.
# - Soil/Urban: Generally increasing reflectance from Blue to SWIR, but variable.

# -----------------------------------------------------------------------------
# Perform Linear Spectral Unmixing
# -----------------------------------------------------------------------------
print("\nPerforming Linear Spectral Unmixing...")

# Use the ee.Image.unmix() function
# Provide the list of endmember spectra.
# Use sumToOne=True to enforce the sum-to-one constraint (fractions sum to 1).
# Use nonNegative=True to enforce the non-negativity constraint (fractions >= 0).
unmixed_image = image_lsu_input.unmix(
    endmembers=endmembers_list,
    sumToOne=True,
    nonNegative=True
)

# The output image ('unmixed_image') will have bands named 'band_0', 'band_1', 'band_2',
# corresponding to the fractional cover of Water, Vegetation, and Soil/Urban respectively,
# because that's the order we supplied them in 'endmembers_list'.

# Check the output band names
unmixed_bands = unmixed_image.bandNames().getInfo()
print("Unmixing complete. Output bands:", unmixed_bands)
# Expected: ['band_0', 'band_1', 'band_2']

# -----------------------------------------------------------------------------
# Visualization
# -----------------------------------------------------------------------------
print("Preparing visualization...")

# 1. Visualization Parameters for RGB
# Use the original composite for a true-color view
rgb_image_display = landsat_composite.select(rgb_bands) # Select RGB bands
rgb_vis_params = {
    'bands': rgb_bands, # R, G, B
    'min': 0.0,
    'max': 0.3, # Typical range for SR reflectance
    'gamma': 1.4
}
print("RGB visualization parameters defined.")

# 2. Visualization Parameters for Unmixed Fractions
# We can display the fractions as an RGB image.
# Let's map: Soil/Urban (band_2) -> Red, Vegetation (band_1) -> Green, Water (band_0) -> Blue
# Fractions range from 0 to 1.
unmix_vis_params = {
    'bands': ['band_2', 'band_1', 'band_0'], # Map fractions to R, G, B
    'min': 0.0, # Minimum fraction is 0
    'max': 1.0  # Maximum fraction is 1
}
print("Unmixed fractions visualization parameters defined (Soil/Urban->R, Veg->G, Water->B).")

# 3. Create a Folium Map
map_lsu = folium.Map(location=map_center_coords, zoom_start=zoom_level)
print(f"Folium map initialized, centered at {map_center_coords} with zoom level {zoom_level}.")

# 4. Add Layers to the Map
# Add the True Color SR image as a base layer
map_lsu = add_ee_layer(map_lsu, rgb_image_display, rgb_vis_params, 'Landsat 8 SR (True Color)')

# Add the Unmixed Fractions layer
map_lsu = add_ee_layer(map_lsu, unmixed_image, unmix_vis_params, 'LSU Fractions (Soil/Urb-R, Veg-G, Wat-B)')

# 5. Add Layer Control
map_lsu.add_child(folium.LayerControl())
print("Added Layer Control to the map.")

Libraries imported and CORRECTED helper function defined.
Libraries imported and helper function defined.
Earth Engine initialization failed: ee.Initialize: no project found. Call with project= or see http://goo.gle/ee-auth.
Attempting authentication...
Google Earth Engine authenticated and initialized successfully.
AOI defined for San Francisco Bay area.
Time period set from 2022-08-01 to 2022-09-30.
Using Landsat Collection: LANDSAT/LC08/C02/T1_L2
Bands for LSU: ['SR_B2', 'SR_B3', 'SR_B4', 'SR_B5', 'SR_B6', 'SR_B7']
Bands for RGB: ['SR_B4', 'SR_B3', 'SR_B2']
Loading and preparing image...
Input image prepared with bands: ['SR_B2', 'SR_B3', 'SR_B4', 'SR_B5', 'SR_B6', 'SR_B7']
Defining endmember locations...
Water endmember point selected.
Vegetation endmember point selected.
Soil/Urban endmember point selected.
Extracting endmember spectra...

Endmember Spectra Extracted (Order: Water, Vegetation, Soil/Urban):
- Water: ['0.0115', '0.0247', '0.0111', '-0.0017', '0.0011', '0.0014']
- Ve

In [3]:
# 6. Display the Map
print("Displaying the interactive map...")
print("Interpretation Guide:")
print("- Areas appearing RED likely have high Soil/Urban fraction.")
print("- Areas appearing GREEN likely have high Vegetation fraction.")
print("- Areas appearing BLUE likely have high Water fraction.")
print("- Mixed colors (Yellow, Cyan, Magenta, White) indicate mixtures of endmembers.")

display(map_lsu)

Displaying the interactive map...
Interpretation Guide:
- Areas appearing RED likely have high Soil/Urban fraction.
- Areas appearing GREEN likely have high Vegetation fraction.
- Areas appearing BLUE likely have high Water fraction.
- Mixed colors (Yellow, Cyan, Magenta, White) indicate mixtures of endmembers.
